In [ ]:
%matplotlib inline
# import matplotlib.image as mpimg
# from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from tf_aerial_images import *
from sklearn import linear_model
from types import SimpleNamespace 
%load_ext autoreload
%autoreload 2

## 1. Load data

In [ ]:
# Loaded a set of images
n = 20

imgs, gt_imgs = load_images(n)

imgs[0].shape, gt_imgs[0].shape

In [ ]:
# # Show first image and its groundtruth image
# cimg = concatenate_images(imgs[1], gt_imgs[1])
# fig1 = plt.figure(figsize=(20, 10))
# plt.imshow(cimg, cmap='Greys_r', vmin=0, vmax=1)

## 2. Generate the input and extract the outputs.
From each image extract the inputs (patches) and generate a list of features for each one of them.

In [ ]:
t = 19 # number of images to use fore the training
train = SimpleNamespace()
train.X = imgs_to_inputs(imgs[:t])
train.Y = imgs_to_outputs(gt_imgs[:t])
train.indices = range(t) # indices on the trained images

test = SimpleNamespace()
test.X = imgs_to_inputs(imgs[t:])
test.Y = imgs_to_outputs(gt_imgs[t:])
test.indices = range(t, n)

train.X.shape, train.Y.shape

Just check the **output** has been generated correctly.

In [ ]:
# Print feature statistics
Y0 = np.where(train.Y==0)[0]
Y1 = np.where(train.Y==1)[0]
print(str(len(Y1)) + ' inputs are classified as background (0)')
print(str(len(Y0)) + ' inputs are classified as road (1)')

## 3. Train a model

In [ ]:
print('Confusion matrix:')
confmat = pd.DataFrame(
    data = [['True Negative (TN)', 'False Negative (FN)'], ['False Positive (FP)', 'True Positive (TP)']],
    index = pd.MultiIndex(
        levels=[['predicted (Z)'], ['0', '1']],
        labels=[[0, 0], [0, 1]]),
    columns = pd.MultiIndex(
        levels=[['actual (Y)'], ['0 (background)', '1 (road)']],
        labels=[[0, 0], [0, 1]]),
)
display(confmat)
print('TN + FN + FP + TP = number of predictions')

- Logistic regression:

In [ ]:
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
# train
logreg.fit(train.X, train.Y)
# predict
train.Z = logreg.predict(train.X)

# print('True positive rate = ' + str(TPR) + ' (#correctly predicted road patches / #predicted road patches)')
display(confusion_matrix(train.Z, train.Y))
print('Lot of non road patches are classified as road')

## 4. Validation

In [ ]:
test.Z = logreg.predict(test.X)

In [ ]:
# Display prediction as an image
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 

patch_size = 16
w, h = 400, 400

# show an image of the test set and the corresponding predictions
i = 0
predicted_im = label_to_img(w, h, patch_size, patch_size, test.Z)
new_img = make_img_overlay(imgs[test.indices[i]], predicted_im)

plt.imshow(new_img)